In [4]:
import sys
import dotenv
import os
dotenv.load_dotenv()
sys.path.append(os.getenv('MAINDIR'))
from main import MoviesDatabase
from sqlalchemy.orm import lazyload, subqueryload, joinedload, selectinload, noload

In [157]:
synopsis = [movie.description for movie in MoviesDatabase.query.all()]
genres = [movie.genre for movie in MoviesDatabase.query.all()]
titles = [movie.title for movie in MoviesDatabase.query.all()]
collection = [movie.collection for movie in MoviesDatabase.query.all()]
critics = [movie.computed_critic_score for movie in MoviesDatabase.query.all()]
studio = [movie.studio for movie in MoviesDatabase.query.all()]

In [6]:
synopsis[0]

"Imprisoned on the other side of the universe, the mighty Thor finds himself in a deadly gladiatorial contest that pits him against the Hulk, his former ally and fellow Avenger. Thor's quest for survival leads him in a race against time to prevent the all-powerful Hela from destroying his home world and the Asgardian civilization."

In [7]:
from transformers import AutoTokenizer, AutoModel
from bert_transformer import BertTransformer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained("bert-base-uncased").to('cuda:0')

bert_transformer = BertTransformer(tokenizer, model, 500)

/home/ksaff/miniconda3/envs/movie_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
vectorized = bert_transformer.fit_transform(synopsis)

/home/ksaff/Desktop/Movie_Base_And_Recommendations/synopsis_clustering/bert_transformer.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(tokenized_text).unsqueeze(0).to(self.device),


In [169]:
from sklearn.cluster import KMeans
kmeans = KMeans(20).fit(vectorized)

/home/ksaff/miniconda3/envs/movie_env/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [170]:
kmeans.labels_

array([16, 14, 11, ...,  2, 10,  2], dtype=int32)

In [171]:
import numpy as np

np.asarray(synopsis).shape

(9887,)

In [172]:
import pandas as pd

data = {'Cluster': kmeans.labels_, 'Synopsis': synopsis, 'Genres': genres, 'Title': titles, 'Collection': collection, 'Critics': critics, 'Studio': studio}
df = pd.DataFrame(data)

In [173]:
df.Studio.value_counts()

Studio
                                                                    2779
Paramount Pictures                                                   405
Warner Bros. Pictures                                                391
Universal Pictures                                                   377
20th Century Fox                                                     373
                                                                    ... 
Warner Bros. Pictures, MGM/UA Home Entertainment Inc.                  1
Roadshow Home Video [au], Metro-Goldwyn-Mayer Distributing Corp.       1
Brandon Films Inc., Cowboy Pictures                                    1
Cinelou Films                                                          1
Paramount+                                                             1
Name: count, Length: 1077, dtype: int64

In [174]:
df[df['Cluster'] == 1].Synopsis

8       A tale that technically spans two centuries, B...
26      To be young and carefree amid the blue waters ...
42      When it's discovered that the evil Emperor Pal...
61      Mickey Pearson is an American expatriate who b...
74      In "Turning Red", Mei Lee is a confident, dork...
                              ...                        
9820    Ui-gang is a hit man who has decided to retire...
9826    A down-on-his-luck hound finds himself in a to...
9846    After more than thirty years of service as one...
9849    Finney, a shy but clever 13-year-old boy, is a...
9853    In the new action comedy from DreamWorks Anima...
Name: Synopsis, Length: 411, dtype: object

In [209]:
n = 4
dict(df[df['Cluster'] == n].Genres.value_counts()[:5]), dict(df[df['Cluster'] == n].Collection.value_counts()[:5]), df[df['Cluster'] == n]['Title'].to_numpy()

({'Drama': 91,
  'Comedy, Romance': 73,
  'Comedy': 71,
  'Comedy, Drama': 38,
  'Horror': 34},
 {'': 476,
  'Divergent': 1,
  'Pixar': 1,
  'Friday the 13th': 1,
  'Bridget Jones': 1},
 array(['Fresh', 'CODA', 'The Lost Daughter', 'Hatching',
        'Fifty Shades of Grey', 'Thoroughbreds', 'Mean Girls', 'Nitram',
        'The Ledge', 'Sex and the City 2', 'Black Swan', 'Cruella',
        'La La Land', 'The Valet', 'The High Note', 'Divergent',
        'The Heat', 'The Age of Adaline', 'Juno', 'Inside Out',
        'Little Women', 'Z', 'How Do You Know', 'My Fake Boyfriend',
        "My Heart Can't Beat Unless You Tell It To", "Jennifer's Body",
        'The Girl Next Door', 'Orphan', 'About Time',
        'The Edge of Seventeen', 'The Devil Wears Prada', 'The Visit',
        'Hollywood Stargirl', 'Unhinged', 'Oculus', 'Step Brothers',
        'Brothers', 'The Boy', 'Block Party', 'Angel Eyes',
        'Below Her Mouth', 'Case 39', 'Begin Again', 'The Sixth Sense',
        'Clueless',